In [1]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = (8.0, 8.0)
import CameraNetwork
import CameraNetwork.global_settings as gs
import cv2
import Image
import ImageDraw
import time
from tqdm import tqdm_notebook
import traceback

c:\users\user\documents\code\cameranetwork_git\CameraNetwork\transformation_matrices.py:1899: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


In [2]:
c = CameraNetwork.CLIclient()
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
SERVERS = c.client_instance.servers
print SERVERS

['109', '109L', '110', '110L', '111', '111L', '112', '113L', '114L', '115', '116', '116L', '117L', '118', '118L', '119', '119L', '120', '219', '999']


In [14]:
CAM = 219
DAY = 15
MONTH = 6
HDR = '3'

In [5]:
df = c.query('{}'.format(CAM), '{:02}/{}/2017'.format(MONTH, DAY))[0]
print("Camera {} captured {} images on this day.".format(CAM, df.shape[0]))

Camera 219 captured 1058 images on this day.


In [17]:
JPEG_QUALITY = 100
video = cv2.VideoWriter(
    'video_{}_2017_{:02}_{:02}_{}.avi'.format(CAM, MONTH, DAY, HDR),
    -1, 6, (800, 600)
)

for ind in tqdm_notebook(df.loc[(slice(None), HDR), :].index):
    try:
        img = c.seek('{}'.format(CAM), ind[0], ind[1], JPEG_QUALITY, 301)[0][0]
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        draw.text(
            (0, 0),
            ind[0].time().strftime("%H:%M:%S"),
            (255,255,255)
        )

        video.write(cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
    except Exception, e:
        traceback.print_exc()
        
video.release()

Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\User\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\User\Anaconda2\lib\site-packages\tqdm\_tqdm.py", line 103, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\User\Anaconda2\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [16]:
video.release()